# 1. import packages 导入包

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataclasses import dataclass

torch.manual_seed(1024)


# 2. Define parameters for GPT 定义GPT的一些参数

In [ ]:
@dataclass
class GPTConfig:
    block_size: int = 512
    batch_size: int = 12
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.1
    head_size: int = n_embd // n_head
    # follow gpt2 tokenzier
    vocab_size: int = 50257
